In [ ]:
import os
import torch

os.environ['HF_TOKEN'] = "" # TODO: Set this

In [2]:
!pip install transformers


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/Phi-4-mini-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

Using device: cuda


Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(200064, 3072, padding_idx=199999)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=5120, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_features=20006

In [ ]:
import json 

calc_tools = [{
    "type": "function", 
        "function": {
        "name": "calculator",
        "description": "Performs mathematical computations with support for basic operations and nested expressions",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "description": "The mathematical operation to perform",
                    "type": "str",
                    "enum": ["add", "subtract", "multiply", "divide"]
                },
                "operands": {
                    "description": "List of numbers",
                    "type": "list"
                }
            },
        }
    }
}]

conversation = [
    {"role": "system", "content": "You are a helpful assistant", "tools": json.dumps(calc_tools)},
    {"role": "user", "content": "What is 2 + 2?"}
]

inputs = tokenizer.apply_chat_template(conversation, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
generated_ids = model.generate(inputs, max_new_tokens=1000)
print(tokenizer.decode(generated_ids[0]))

<|system|>You are a helpful assistant<|tool|>[{"type": "function", "function": {"name": "calculator", "description": "Performs mathematical computations with support for basic operations and nested expressions", "parameters": {"type": "object", "properties": {"operation": {"description": "The mathematical operation to perform", "type": "str", "enum": ["add", "subtract", "multiply", "divide"]}, "operands": {"description": "List of numbers", "type": "list"}}}}}]<|/tool|><|end|><|user|>What is 2 + 2?<|end|><|assistant|><|tool_call|>[{"type": "function", "function": {"name": "calculator", "arguments": {"operation": "add", "operands": [2, 2]}}}]<|/tool_call|><|end|>


# End result

This model outputs tool calls in the below format:
`<|assistant|><|tool_call|>[{"type": "function", "function": {"name": "", "arguments": {}}}]<|/tool_call|><|end|>`

Which means the tool parsing will need to handle potentially multiple tool calls.